# Continuation of the King County Real Estate Prediction
### Previously we pickled the final model, and now we will test the prediction on a holdout set. 

In [1]:
import pandas as pd
import pickle
pd.set_option('display.max_columns', 300)

In [2]:
#Importing data
holdout = pd.read_csv("./datasets/kc_house_data_test_features.csv", index_col = 0)
extra = pd.read_csv("./datasets/zip_code_extra_data_fixed.csv", index_col = 0)


In [3]:
# opening pickle
price_prediction_model=pickle.load(open("lm_final.pickle", "rb"))

#### I will now perform the same analysis and feature engeinering that was performed in the training set. 

In [4]:
holdout.head()
holdout_date = holdout[["date"]]
holdout.drop(columns=["date"], inplace = True)

In [5]:
print(holdout.shape) 
holdout.drop(columns=["id"], inplace = True)

(4323, 19)


In [6]:
holdout_date.shape


(4323, 1)

In [7]:
holdout.columns.to_list()

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [8]:
# removes outliers - bedrooms  > 10

def cap_max(x, n, data, column):
    """
    x = the column number
    n = min number
    data = dataframe
    column = interested column from dataframe
    should be applied in a lambda function.
    For example: sales["bedrooms"].apply(lambda x: cutoff(x,10,sales,"bedrooms"))
    """
    if x > n:
        return data[column].median()
    else:
        return x

holdout["bedrooms"]= holdout["bedrooms"].apply(lambda x: cap_max(x,10,holdout,"bedrooms"))  

In [9]:
def cap_min(x, n, data, column):
    """
    x = the column number
    n = min number
    data = dataframe
    column = interested column from dataframe
    should be applied in a lambda function.
    For example: sales["bedrooms"].apply(lambda x: cutoff(x,10,sales,"bedrooms"))
    """
    if x == n:
        return 1
    else:
        return x

holdout["bathrooms"]= holdout["bathrooms"].apply(lambda x: cap_min(x,0,holdout,"bedrooms"))  

In [10]:
holdout.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [11]:
holdout = pd.merge(holdout, extra, how='right', on='zipcode')

In [12]:
holdout.shape

(4323, 72)

In [13]:
holdout_date

,date
0,20140827T000000
1,20150218T000000
2,20141107T000000
3,20141203T000000
4,20150115T000000
...,...
4318,20140521T000000
4319,20150223T000000
4320,20140623T000000
4321,20150116T000000


In [14]:
holdout = pd.merge(holdout, holdout_date, how='left', left_index = True, right_index = True)
holdout["month"]= pd.DatetimeIndex(holdout["date"]).month
holdout.drop(columns=["date"], inplace = True)

In [15]:
holdout.shape

(4323, 73)

In [16]:
holdout.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'full_time_earnings',
       'part_time_earnings', 'no_earnings', 'car_truck_van',
       'public_transportation', 'taxi', 'motorcycle', 'bike_walk_other',
       'work_home', 'less_than_hs', 'high_school_grad', 'associates',
       'bachelors', 'masters', 'professional_school_degree', 'doctorate',
       'public_school_enrollment', 'private_school_enrollment',
       'no_school_enrollment', 'for_rent', 'rented_and_unoccupied',
       'for_sale_only', 'sold_and_unoccupied', 'seasonal_or_rec_use',
       'migrant_worker_housing', 'vacant_other_reasons',
       'in_occupied_housing_units', 'adult_correctional_facility',
       'juvenile_facilities', 'nursing_facilities', 'other_institutional',
       'military_quarters', 'other_noninstitution

In [17]:
from geopy.distance import geodesic
downtown = (47.609862, -122.342056) 
holdout['coords'] = tuple(zip(holdout['lat'],holdout['long']))
x = holdout['coords'].apply(lambda x: geodesic(downtown,x))
holdout['miles_from_downtown'] = x.astype(str).str[:-3].astype(float)
holdout.drop(columns = "coords", inplace = True)

In [18]:
holdout.shape

(4323, 74)

In [19]:
holdout.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'full_time_earnings',
       'part_time_earnings', 'no_earnings', 'car_truck_van',
       'public_transportation', 'taxi', 'motorcycle', 'bike_walk_other',
       'work_home', 'less_than_hs', 'high_school_grad', 'associates',
       'bachelors', 'masters', 'professional_school_degree', 'doctorate',
       'public_school_enrollment', 'private_school_enrollment',
       'no_school_enrollment', 'for_rent', 'rented_and_unoccupied',
       'for_sale_only', 'sold_and_unoccupied', 'seasonal_or_rec_use',
       'migrant_worker_housing', 'vacant_other_reasons',
       'in_occupied_housing_units', 'adult_correctional_facility',
       'juvenile_facilities', 'nursing_facilities', 'other_institutional',
       'military_quarters', 'other_noninstitution

In [20]:
holdout["house_years"] = abs(holdout["yr_renovated"]-holdout["yr_built"])
holdout["house_age_2015"] = 2015 - holdout["yr_built"]


In [21]:
holdout.drop(columns= ["yr_built", "yr_renovated"], inplace = True)

In [22]:
holdout.shape

(4323, 74)

In [23]:
zipcode_dummies_holdout = pd.get_dummies(holdout["zipcode"], prefix="zipcode",  drop_first = True)


In [24]:
holdout = pd.concat([holdout, zipcode_dummies_holdout], axis=1)

In [25]:
print(holdout.shape)

(4323, 143)


In [26]:
data_fin = pd.read_csv("final_data.csv", index_col = 0)

In [27]:
holdout.shape

(4323, 143)

In [28]:
data_fin.shape

(17290, 100)

In [29]:
list(set(holdout.columns.to_list()) - set(data_fin.columns.to_list()))

['zipcode_98115',
 'zipcode_98070',
 'motorcycle',
 'zipcode_98065',
 'zipcode_98007',
 'zipcode_98136',
 'miles_from_downtown',
 'month',
 'zipcode_98010',
 'zipcode_98045',
 'zipcode_98117',
 'zipcode_98028',
 'zipcode_98103',
 'long',
 'other_noninstitutional',
 'zipcode_98148',
 'zipcode_98125',
 'zipcode_98029',
 'zipcode_98072',
 'zipcode_98166',
 'zipcode_98144',
 'zipcode_98059',
 'total_housing_units',
 'taxi',
 'condition',
 'juvenile_facilities',
 'zipcode_98034',
 'zipcode_98155',
 'renter_occupied_housing',
 'zipcode_98116',
 'zipcode_98008',
 'zipcode_98019',
 'zipcode_98024',
 'house_owned_outright',
 'zipcode_98056',
 'zipcode_98011',
 'zipcode_98177',
 'zipcode_98077',
 'zipcode_98027',
 'zipcode_98107',
 'zipcode_98122',
 'zipcode_98126',
 'zipcode_98014']

In [30]:
holdout[data_fin.columns]

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,sqft_basement,zipcode,lat,sqft_living15,sqft_lot15,full_time_earnings,part_time_earnings,no_earnings,car_truck_van,public_transportation,bike_walk_other,work_home,less_than_hs,high_school_grad,associates,bachelors,masters,professional_school_degree,doctorate,public_school_enrollment,private_school_enrollment,no_school_enrollment,for_rent,rented_and_unoccupied,for_sale_only,sold_and_unoccupied,seasonal_or_rec_use,migrant_worker_housing,vacant_other_reasons,in_occupied_housing_units,adult_correctional_facility,nursing_facilities,other_institutional,military_quarters,house_owned_with_mortgage,vacant_households,husband_and_wife_house,single_guardian,singles,singles_with_roommate,male,female,median_age,male_median_age,female_median_age,population,population_density_per_sq_mile,median_home_value,land_area,water_area_sq_mile,occupied_housing_units,median_household_income,house_years,house_age_2015,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98022,zipcode_98023,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98058,zipcode_98074,zipcode_98075,zipcode_98092,zipcode_98102,zipcode_98105,zipcode_98106,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98118,zipcode_98119,zipcode_98133,zipcode_98146,zipcode_98168,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4,2.50,2270,11500,1.0,0,0,8,1540,730,98034,47.7089,2020,10918,15037,8371,7539,17063,1553,514,1132,1624,10016,2577,8570,3402,658,436,4764,1145,639,463,28,221,26,71,1,179,39878,0,165,4,0,8562,989,7893,2246,5017,1735,20017,20390,36,35,38,40407,4428,364900.0,9.13,1.69,16891,77688.0,1967,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2.50,2270,11500,1.0,0,0,8,1540,730,98034,47.7089,2020,10918,15037,8371,7539,17063,1553,514,1132,1624,10016,2577,8570,3402,658,436,4764,1145,639,463,28,221,26,71,1,179,39878,0,165,4,0,8562,989,7893,2246,5017,1735,20017,20390,36,35,38,40407,4428,364900.0,9.13,1.69,16891,77688.0,1967,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2.50,2820,14890,1.0,0,4,9,1770,1050,98034,47.7019,3740,14890,15037,8371,7539,17063,1553,514,1132,1624,10016,2577,8570,3402,658,436,4764,1145,639,463,28,221,26,71,1,179,39878,0,165,4,0,8562,989,7893,2246,5017,1735,20017,20390,36,35,38,40407,4428,364900.0,9.13,1.69,16891,77688.0,1985,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2.25,1600,10240,1.0,0,0,7,1090,510,98034,47.7281,1520,9394,15037,8371,7539,17063,1553,514,1132,1624,10016,2577,8570,3402,658,436,4764,1145,639,463,28,221,26,71,1,179,39878,0,165,4,0,8562,989,7893,2246,5017,1735,20017,20390,36,35,38,40407,4428,364900.0,9.13,1.69,16891,77688.0,1979,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,2.50,2300,4307,2.0,0,0,8,2300,0,98034,47.7326,2010,4307,15037,8371,7539,17063,1553,514,1132,1624,10016,2577,8570,3402,658,436,4764,1145,639,463,28,221,26,71,1,179,39878,0,165,4,0,8562,989,7893,2246,5017,1735,20017,20390,36,35,38,40407,4428,364900.0,9.13,1.69,16891,77688.0,1998,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,6,2.75,3970,9500,1.0,0,0,10,2180,1790,98005,47.5956,2490,9775,6500,3307,4636,6461,1032,389,524,454,2954,875,4743,2368,500,412,2148,566,185,187,13,89,29,140,0,73,17549,0,0,0,0,3193,531,3800,781,2286,672,8930,8784,38,36,40,17714,2361,568700.0,7.50,0.01,7539,84774.0,1970,45,0,0,0

In [31]:
pickle_in = open("lm_final.pickle", "rb")
modle = pickle.load(pickle_in)
pickle_in.close()

In [32]:
y_hat = modle.predict(holdout[data_fin.columns])

In [33]:
pd.DataFrame(y_hat, columns = ["price_pred"]).to_csv("housing_preds_dorjey_sherpa")